In [1]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

fatal: destination path 'TACL2015' already exists and is not an empty directory.
fatal: destination path 'nlu-asdiv-dataset' already exists and is not an empty directory.
fatal: destination path 'grade-school-math' already exists and is not an empty directory.


In [1]:
import os
from xml.etree import ElementTree
import numpy as np
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import StringIO
from contextlib import redirect_stdout
from termcolor import colored

In [2]:
import dataset_handler as dh
import helper_func as hf

In [3]:
torch.manual_seed(0)
np.random.seed(8)

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

"""Load the priming text to add to the prompt and sample a question"""
priming_text_path = "data/priming_texts/gsm8k/gsm8k_fewer_alt_codegen.txt"
# priming_text = read_string_from_file("data/priming_texts/singleEq.txt")
# priming_text_path = "data/priming_texts/asdiv/asdiv_prefix_codegen.txt"

current_dataset = dh.init_dataset_from_name("gsm8k", primingtext_path = priming_text_path)

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(10)

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

Write a program that prints the answer to the following question. Carol sells tickets for an exhibition. During three days she sold tickets worth $960. One ticket costs $4. How many tickets on average did she sell during one of these three days?
80


In [4]:
"""CodeGen runs in the venv venv"""
model_args = hf.model_args()
model, tokenizer = hf.load_CodeGen(model_args)

loading parameters
loading parameters took 433.48s
loading tokenizer
loading tokenizer took 4.08s


In [ ]:
"""GPT-J and codeparrot models run in HFTest venv"""
tokenizer = AutoTokenizer.from_pretrained(gptj_model)
model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()

In [5]:
torch.manual_seed(42)
np.random.seed(42)

n = 3
k = 3
"""n = 4
k = 3"""

gen_args = hf.codegen_gen_args()
gen_args.num_return_sequences = n
gen_args.k = k

hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, gen_args)

TESTING STARTED
@sample 5 -> Pass@3 = 0.2
@sample 10 -> Pass@3 = 0.2


Pass@3 = 0.2


0.2